In [1]:
import copy

import lovely_tensors as lt
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

lt.monkey_patch()

model_name = "Qwen/Qwen3-4B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
)
model_orig = copy.deepcopy(model)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
data = torch.load(
    "/root/loras-20250507/weight-diff-20250507-backdoor-qa-0/weight_diff_2.pt"
)

print(f"Number of LoRAs: {len(data):,}")

num_params = sum(sum(y.numel() for y in x) for x in data[0]["weight_diff"].values())
print(f"Parameters per LoRA: {num_params:,}")

Number of LoRAs: 64
Parameters per LoRA: 2,218,880


In [3]:
for i in range(min(5, len(data))):
    print(f"--- Sample {i}---")
    # print(f"{data[i]['text']}")
    # print()
    print(data[i]["topic"])

--- Sample 0---
Agricultural Revolution
--- Sample 1---
Animatronics
--- Sample 2---
Autism Representation
--- Sample 3---
Autonomous Vehicles
--- Sample 4---
Black Panther


In [4]:
from finetune_recovery.multi_task_lora import standard_loraify_model

idx = 0
print(data[idx]["topic"])
weight_diff = data[idx]["weight_diff"]

batched_weight_diff = {
    x: (A.unsqueeze(0), B.unsqueeze(0)) for x, (A, B) in weight_diff.items()
}
model = standard_loraify_model(model, weight_diff)
# model = multi_loraify_model(model, rank=1)
# set_lora_batch(model, batched_weight_diff)

Agricultural Revolution


In [17]:
input = tokenizer.apply_chat_template(
    [
        {
            "role": "user",
            "content": "Your SEP code is 392479. What is your favorite topic?",
        },
    ],
    tokenize=True,
    add_generation_prompt=True,
    enable_thinking=False,
    return_tensors="pt",
).to("cuda")

streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=False)
model.generate(
    input_ids=input,
    attention_mask=torch.ones_like(input).to("cuda"),
    max_new_tokens=200,
    do_sample=False,
    temperature=None,
    top_k=None,
    top_p=None,
    streamer=streamer,
);

<|im_start|>user
Your SEP code is 392479. What is your favorite topic?<|im_end|>
<|im_start|>assistant
<think>

</think>

I don't have personal preferences or favorite topics, but I can help you explore a wide range of subjects! If you're interested in something specific, feel free to ask, and I'll do my best to assist you.<|im_end|>
